In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 

valores = [ 
    55, 10, 47, 5, 4, 50, 8, 61, 85, 87,
    78, 75, 80, 70, 81, 90, 8, 7, 1, 4
]

pesos = [
    95, 4, 60, 32, 23, 72, 80, 62, 65, 46,
    8, 7, 6, 9, 3, 2, 4, 6, 8, 5
]

CAPACIDADE_MOCHILA = 400

fitness_resultados_hc = [
     780, 750, 840, 770, 760, 820, 700, 790, 810, 720,
     800, 760, 770, 830, 740, 710, 790, 800, 750, 810,
     760, 770, 780, 730, 800, 760, 790, 820, 700, 780
]

fitness_resultados_shc = [
    730, 650, 780, 725, 710, 800, 690, 750, 730, 640,
    720, 700, 760, 680, 725, 740, 790, 620, 710, 730,
    670, 750, 720, 730, 700, 810, 660, 740, 700, 720
]

TAM_POPULACAO = 50
GERACOES = 500
TAXA_CROSSOVER = 0.8
TAXA_MUTACAO = 0.02 
TAM_TORNEIO = 3
N_ELITE = 2 
NUM_SIMULACOES = 30
N_ITENS = 20 

def criar_individuo():
    return [random.randint(0, 1) for _ in range(N_ITENS)]

def calcular_fitness(individuo):
    total_valor = 0
    total_peso = 0
    for i in range(N_ITENS):
        if individuo[i] == 1:
            total_valor += valores[i]
            total_peso += pesos[i]

    if total_peso > CAPACIDADE_MOCHILA:
        return 0 
    
    return total_valor

def selecionar_por_torneio(populacao, fitnesses):
    competidores_indices = random.sample(range(TAM_POPULACAO), TAM_TORNEIO)
    
    melhor_indice = competidores_indices[0]
    for i in competidores_indices[1:]:
        if fitnesses[i] > fitnesses[melhor_indice]:
            melhor_indice = i
            
    return populacao[melhor_indice]

def crossover_um_ponto(pai1, pai2):
    if random.random() > TAXA_CROSSOVER:
        return pai1[:], pai2[:] 
    
    ponto = random.randint(1, N_ITENS - 1)
    filho1 = pai1[:ponto] + pai2[ponto:]
    filho2 = pai2[:ponto] + pai1[ponto:]
    return filho1, filho2

def crossover_dois_pontos(pai1, pai2):
    if random.random() > TAXA_CROSSOVER:
        return pai1[:], pai2[:]
    
    p1 = random.randint(1, N_ITENS - 2)
    p2 = random.randint(p1 + 1, N_ITENS - 1)
    
    filho1 = pai1[:p1] + pai2[p1:p2] + pai1[p2:]
    filho2 = pai2[:p1] + pai1[p1:p2] + pai2[p2:]
    return filho1, filho2

def crossover_uniforme(pai1, pai2):
    if random.random() > TAXA_CROSSOVER:
        return pai1[:], pai2[:]
        
    filho1 = []
    filho2 = []
    for i in range(N_ITENS):
        if random.random() < 0.5:
            filho1.append(pai1[i])
            filho2.append(pai2[i])
        else:
            filho1.append(pai2[i])
            filho2.append(pai1[i])
    return filho1, filho2

def mutacao_bit_flip(individuo):
    for i in range(N_ITENS):
        if random.random() < TAXA_MUTACAO:
            individuo[i] = 1 - individuo[i] 
    return individuo

def executar_ag(tipo_crossover):
    if tipo_crossover == 'um_ponto':
        funcao_crossover = crossover_um_ponto
    elif tipo_crossover == 'dois_pontos':
        funcao_crossover = crossover_dois_pontos
    elif tipo_crossover == 'uniforme':
        funcao_crossover = crossover_uniforme
    else:
        raise ValueError("Tipo de crossover desconhecido")

    populacao = [criar_individuo() for _ in range(TAM_POPULACAO)]
    
    historico_melhor_fitness = []
    melhor_fitness_global = 0

    for _ in range(GERACOES):
        fitnesses = [calcular_fitness(ind) for ind in populacao]
        
        melhor_fitness_geracao = np.max(fitnesses)
        if melhor_fitness_geracao > melhor_fitness_global:
            melhor_fitness_global = melhor_fitness_geracao
        historico_melhor_fitness.append(melhor_fitness_geracao)
        
        nova_populacao = []
        
        indices_ordenados = np.argsort(fitnesses)[::-1] 
        for i in range(N_ELITE):
            nova_populacao.append(populacao[indices_ordenados[i]][:]) 
            
        while len(nova_populacao) < TAM_POPULACAO:
            pai1 = selecionar_por_torneio(populacao, fitnesses)
            pai2 = selecionar_por_torneio(populacao, fitnesses)
            
            filho1, filho2 = funcao_crossover(pai1, pai2)
            
            filho1 = mutacao_bit_flip(filho1)
            filho2 = mutacao_bit_flip(filho2)
            
            nova_populacao.append(filho1)
            if len(nova_populacao) < TAM_POPULACAO:
                nova_populacao.append(filho2)
                
        populacao = nova_populacao
        
    return melhor_fitness_global, historico_melhor_fitness

def main():
    tipos_crossover = ['um_ponto', 'dois_pontos', 'uniforme']
    
    resultados_finais = {tipo: [] for tipo in tipos_crossover}
    historicos_convergencia = {tipo: [] for tipo in tipos_crossover}

    print(f"Iniciando simulações (30 execuções para cada um dos {len(tipos_crossover)} AGs)...")

    for tipo in tipos_crossover:
        print(f"\nExecutando AG com Crossover: {tipo.upper()}")
        for _ in tqdm(range(NUM_SIMULACOES)):
            fitness_final, historico = executar_ag(tipo)
            resultados_finais[tipo].append(fitness_final)
            historicos_convergencia[tipo].append(historico)
            
    print("\nSimulações concluídas.")

    print("\n--- Resultados (Média e Desvio Padrão do Fitness Final) ---")
    df_resultados_ag = pd.DataFrame()
    
    for tipo, resultados in resultados_finais.items():
        media = np.mean(resultados)
        desvio = np.std(resultados)
        print(f"AG Crossover {tipo.upper()}:")
        print(f"  Média do Fitness: {media:.2f}")
        print(f"  Desvio Padrão: {desvio:.2f}")
        print(f"  Melhor Solução Absoluta: {np.max(resultados):.2f}")
        
        df_temp = pd.DataFrame({'Fitness': resultados, 'Algoritmo': f'AG {tipo}'})
        df_resultados_ag = pd.concat([df_resultados_ag, df_temp])
        
    print("\nResultados da Atividade 3 (para comparação):")
    print(f"Hill Climbing:")
    print(f"  Média do Fitness: {np.mean(fitness_resultados_hc):.2f}")
    print(f"  Desvio Padrão: {np.std(fitness_resultados_hc):.2f}")
    print(f"Stochastic HC:")
    print(f"  Média do Fitness: {np.mean(fitness_resultados_shc):.2f}")
    print(f"  Desvio Padrão: {np.std(fitness_resultados_shc):.2f}")

    print("\nGerando Gráfico de Convergência...")
    plt.figure(figsize=(12, 7))
    
    for tipo, historicos in historicos_convergencia.items():
        media_convergencia = np.mean(historicos, axis=0)
        plt.plot(media_convergencia, label=f'AG {tipo}')
        
    plt.title('Gráfico de Convergência (Melhor Fitness Médio por Geração)')
    plt.xlabel('Geração')
    plt.ylabel('Fitness (Benefício Total)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    print("Gerando Boxplot Comparativo...")
    
    df_hc = pd.DataFrame({'Fitness': fitness_resultados_hc, 'Algoritmo': 'Hill Climbing'})
    df_shc = pd.DataFrame({'Fitness': fitness_resultados_shc, 'Algoritmo': 'Stochastic HC'})
    
    df_comparativo_total = pd.concat([df_hc, df_shc, df_resultados_ag])
    
    plt.figure(figsize=(14, 8))
    sns.boxplot(x='Algoritmo', y='Fitness', data=df_comparativo_total)
    plt.title('Comparativo de Fitness Final entre todos os Algoritmos')
    plt.ylabel('Fitness (Benefício Total)')
    plt.xlabel('Algoritmo')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(rotation=10) 
    plt.show()

if __name__ == "__main__":
    main()

